image analysis running on the server "lev", sharing the basic same data.  presently, the IA port is not exposed, but in interest of time, i am making a "pass-through" access to the IA api via a WB api call.  (this is ultimately the model we want to use in production, so that wildbook can handle security and other issues, but as you will see, i have kind of done a very generic hack version of the same concept.)   this consists of a small wb api "wrapper" call around the call you actually wish to be making of the IA data.  perhaps examples would demonstrate best:

IA call:  /api/annot/image/contributor/tag/json/?annot_uuid_list=[{"__UUID__":"8b595dc0-9c5a-4caf-9703-9f8ff017e824"}]
becomes: http://lev.cs.rpi.edu:8080/ggr/ia?passthru=/api/annot/image/contributor/tag/json/&arg=annot_uuid_list%3D[{%22__UUID__%22:%228b595dc0-9c5a-4caf-9703-9f8ff017e824%22}]

IA call:  /api/annot/age/months/json/?annot_uuid_list=[{"__UUID__":"8b595dc0-9c5a-4caf-9703-9f8ff017e824"}]
becomes:  http://lev.cs.rpi.edu:8080/ggr/ia?passthru=/api/annot/age/months/json/&arg=annot_uuid_list%3D[{%22__UUID__%22:%228b595dc0-9c5a-4caf-9703-9f8ff017e824%22}]

in other words, you pass two parameters, passthru and arg, which are just uri-encoded strings that represent the two sides of the "?" in the original call.  (note that arg is optional.)  the two examples above are "live", in the sense that you can click them and should get the json results as expected.

this should, technically, get you to any existing IA api call (provided i had jason p turn on all the right ones)... at least the ones that use GET.  if you need any POST queries, let me know and i can pass those through as well.

In [18]:
import requests
import urllib
import GetPropertiesAPI as GP
from collections import OrderedDict
import importlib
importlib.reload(GP)
DOMAIN = 'http://lev.cs.rpi.edu:8080/ggr/ia'

In [ ]:
## /api/annot/image/contributor/tag/json/?annot_uuid_list=[{"__UUID__":"8b595dc0-9c5a-4caf-9703-9f8ff017e824"}]


param = urllib.parse.quote('annot_uuid_list=') + '[{' + urllib.parse.quote('"__UUID__"') + ':' +urllib.parse.quote('"8b595dc0-9c5a-4caf-9703-9f8ff017e824"') + '}]'
# param = urllib.parse.urlencode(param) 

In [ ]:
payload = OrderedDict([('passthru', u'/api/annot/age/months/json/'), 
                ('asdasarg', param)]) 
response = requests.get(DOMAIN , 
                        params = payload 
                        )

In [8]:
url = DOMAIN + '?passthru=' +"/api/annot/age/months/json" + '&arg=' + GP.ggr_annot_form_arg('3f814994-216c-4015-9af3-f286481414c0')

In [9]:
res = requests.get(url)
# "/api/annot/age/months/json" - age
# "/api/annot/sex/json/" - sex
# "/api/annot/name/rowid/json/" - NID
# 

In [10]:
res.json()

{'results': [[None, 2]], 'success': True}

In [ ]:
url = DOMAIN + '?passthru=' +"/api/annot/age/months/json/" + '&arg=' + ggxr_form_arg("8b595dc0-9c5a-4caf-9703-9f8ff017e824")

In [ ]:
s = "abcd"

'\"%s\"' %s

In [ ]:
ggr_form_arg = lambda x : urllib.parse.quote('annot_uuid_list=') + '[{' + urllib.parse.quote('"__UUID__"') + ':' + urllib.parse.quote('\"%s\"' %x) + '}]'

In [ ]:
ggr_image_form_arg = lambda x : urllib.parse.quote('image_uuid_list=') + '[{' + urllib.parse.quote('"__UUID__"') + ':' + urllib.parse.quote('\"%s\"' %x) + '}]'

In [68]:
import UploadAndDetectIBEIS as UD, uuid
import requests, json, datetime, urllib, importlib
importlib.reload(UD)

import GetPropertiesAPI as GP
importlib.reload(GP)

<module 'GetPropertiesAPI' from '/Users/sreejithmenon/Google Drive/CodeBase/AWESOME/script/GetPropertiesAPI.py'>

In [ ]:
data_dict = {
        'gid_list': [1],
    }
image_uuid_list = UD.get('api/image/uuid', data_dict)
image_uuid_dict = image_uuid_list[0]
image_uuid = uuid.UUID(image_uuid_dict['__UUID__'])

data_dict = {
    'query_annot_uuid_list' : [1],
    'query_annot_name_list' : [image_uuid]
    
}
res = requests.get('http://pachy.cs.uic.edu:5000/api/engine/query/graph/', data = data_dict)
res.json()

In [ ]:
data_dict = {
        'aid_list': [1],
    }
image_uuid_list = UD.get('api/annot/uuid', data_dict)
image_uuid_list

In [ ]:
res = requests.get('http://pachy.cs.uic.edu:5001/api/annot/uuid/', 
                   data = dict(aid_list=['1']))


print(res.json())

res = requests.get('http://pachy.cs.uic.edu:5000/api/annot/uuid/', 
                   data = dict(aid_list=['15441']))


print(res.json())

In [ ]:
url = "http://pachy.cs.uic.edu:5001/api/engine/query/graph/"
data_dict = {
    'query_annot_uuid_list' : json.dumps([{'__UUID__': '2637b5e7-2a41-4aea-9dbd-dfe186cc8c8c'}]),
}
response = requests.request('POST', url, data=data_dict)

In [ ]:
response.json()

In [ ]:
d['json_result']['inference_dict']['cluster_dict']

In [ ]:
d['json_result']['inference_dict']['annot_pair_dict']

In [ ]:
re.findall(r'NEWNAME_(\d+)', 'NEWNAME_-6')

In [16]:
import os, json, re

In [ ]:
data_dict = {
        'gid_list': list(range(1702, 1792)),
    }

UD.delete('api/image',data_dict)

In [33]:
with open("../data/beauty_ggr.json", "r") as bty_ggr_fl:
    bty_ggr= json.load(bty_ggr_fl)
    
keys = list(bty_ggr.keys())

In [16]:
import BuildConsolidatedFeaturesFile as BCF
import importlib
importlib.reload(BCF)
import urllib

In [58]:
uuid_annot_map = BCF.build_feature_file_ggr(None, "/tmp/test")

In [17]:
aid_uuid_list = [item for sublist in list(uuid_annot_map.values()) for item in sublist if item]

In [18]:
aid_uuid_list

['9849a3a0-a8de-4c0a-b5db-4fa690a3578e',
 '146eab19-1ac4-46b0-90d0-72d4471fce76',
 '046bd8aa-8f77-4ae4-8048-04d86063879a',
 'ae0730b9-816f-45fb-ba5d-c45b2fdd79fe',
 'e41701a3-cb30-4739-bbeb-c35f46f40719',
 '48ad9ae8-43f2-499b-8755-b7e155212f61',
 'd66d3dcf-11fd-4a3d-8522-f39e9eb0934b',
 'b40ccf2a-3744-4e12-9a38-bab7311a5a19',
 'fd7d18ba-a66a-4e79-8daa-58e0790b6e99',
 'c47b15cb-3f18-4305-85b5-0d7fe67e02cc',
 'e62eaa9c-6e6c-4ee6-ad64-5ea64a179d6f',
 '34d11736-9ed7-45c8-aa5c-e82103901130']

In [22]:
ggr_image_form_arg = lambda x : urllib.parse.quote('image_uuid_list=') + '[{' + urllib.parse.quote('"__UUID__"') + ':' + urllib.parse.quote('\"%s\"' %x) + '}' + ',{' + urllib.parse.quote('"__UUID__"') + ':' + urllib.parse.quote('\"%s\"' %x) + '}]'

In [49]:
GP.ggr_image_form_arg('9849a3a0-a8de-4c0a-b5db-4fa690a3578e')

'image_uuid_list%3D[{%22__UUID__%22:%229849a3a0-a8de-4c0a-b5db-4fa690a3578e%22}]'

In [27]:
ggr_image_form_arg = lambda x : urllib.parse.quote('image_uuid_list=') + '[{' + urllib.parse.quote('"__UUID__"') + ':' + urllib.parse.quote('\"%s\"' %x) + '}]'

In [46]:
ggr_image_form_arg('9849a3a0-a8de-4c0a-b5db-4fa690a3578e')

data = json.dumps({
"passthru" : "/api/image/annot/uuid", 
"arg" : {
    'image_uuid_list' : json.dumps([{"__UUID__" : "9849a3a0-a8de-4c0a-b5db-4fa690a3578e" }])
}
})
req = requests.request("GET", url=DOMAIN, data=data)

In [16]:
ggr_contrib_form_arg = lambda x : urllib.parse.quote('contributor_tag_list=') + '[' + urllib.parse.quote('\"%s\"' %x) + ']'

In [14]:
import json

DOMAIN + "?passthru=/api/contributor/gids/unixtime/?gid_list="+ json.dumps([1])

GP.ggr_get("/api/image/note/?gid_list=[1]")


{'results': ['GGR,100,A,00000'], 'success': True}

In [18]:
# GGR.get("/api/contributor/gids/unixtime")


ggr_contrib_form_arg("GGR,100,A,00000")


'contributor_tag_list%3D[%22GGR%2C100%2CA%2C00000%22]'

In [19]:
GP.ggr_annot_form_arg('blah')

'annot_uuid_list%3D[{%22__UUID__%22:%22blah%22}]'

In [22]:
url = DOMAIN + "?passthru=" + "/api/contributor/gids/unixtime" + "&arg=" + "contributor_tag_list%3D[%22GGR%2C100%2CA%2C00000%22]"

In [23]:
url

'http://lev.cs.rpi.edu:8080/ggr/ia?passthru=/api/contributor/gids/unixtime&arg=contributor_tag_list%3D[%22GGR%2C100%2CA%2C00000%22]'

In [24]:
# 45775b50-cb67-51d0-a6a6-3dd88f4fbde2

GP.ggr_get("/api/image/lon", GP.ggr_image_form_arg("45775b50-cb67-51d0-a6a6-3dd88f4fbde2"))

{'exception': 'java.lang.RuntimeException: Failed : HTTP error code : 500',
 'success': False}

In [34]:
import re

keys = [re.findall(r'(.*).jpg', key)[0] for key in keys]

key_dash = keys[:10]

In [43]:
uuid_gid_map = {GP.ggr_get("/api/image/uuid/?gid_list=[%i]" %gid)['results'][0]["__UUID__"] : gid for gid in range(1,37434)}

In [40]:
for i in range(1,11):
    print(GP.ggr_get("/api/image/uuid/?gid_list=[%i]" %i)['results'][0]["__UUID__"])

5ee9cb4b-b4fa-bec5-cca0-31c341d58730
c861876f-66f4-fec7-f9b5-a97314ba3647
01e3b8d2-a80d-547d-8897-ebb900e00595
85f7c1c4-77fc-8b53-83c3-56bbda156da4
f834d0d9-0fd5-c222-2419-eebb5deb090b
36fc39fa-d931-536a-6a70-4aab322b906c
ca0e8c7d-8758-160a-f089-8df3c208db85
e1f85f87-6f9c-c4d0-330d-f82b81e8ab8e
f611b15d-1e8c-a500-e8ab-d75322da209e
f42ca8c2-f653-0f54-0653-3514e6ea1e1b


In [52]:
with open("/tmp/uuid_gid_map.json", "w") as fl:
    json.dump(uuid_gid_map, fl, indent=4)


uuid_gid_map

{'01e3b8d2-a80d-547d-8897-ebb900e00595': 3,
 '36fc39fa-d931-536a-6a70-4aab322b906c': 6,
 '5ee9cb4b-b4fa-bec5-cca0-31c341d58730': 1,
 '85f7c1c4-77fc-8b53-83c3-56bbda156da4': 4,
 'c861876f-66f4-fec7-f9b5-a97314ba3647': 2,
 'ca0e8c7d-8758-160a-f089-8df3c208db85': 7,
 'e1f85f87-6f9c-c4d0-330d-f82b81e8ab8e': 8,
 'f42ca8c2-f653-0f54-0653-3514e6ea1e1b': 10,
 'f611b15d-1e8c-a500-e8ab-d75322da209e': 9,
 'f834d0d9-0fd5-c222-2419-eebb5deb090b': 5}

In [45]:
ggr_gid_form_arg = lambda x : urllib.parse.quote('gid_list=') + '[' + urllib.parse.quote('\"%s\"' %x)  + ']'

In [47]:
ggr_gid_form_arg("1")

'gid_list%3D[%221%22]'

In [49]:
lat = GP.ggr_get("/api/image/lat", GP.ggr_gid_form_arg(1))

In [50]:
lat

{'results': [0.27842], 'success': True}

In [17]:
BCF.build_exif_ftrs_fl_ggr("../data/uuid_gid_map.json", 
                           "../data/ggr_uuid_list.dat",
                           "/tmp/test_exif.json", 1, 10)

100 seconds elapsed..!
100 seconds elapsed..!
100 seconds elapsed..!


0

In [57]:
len(keys)

37433

In [58]:
str(u'01e3b8d2-a80d-547d-8897-ebb900e00595')

'01e3b8d2-a80d-547d-8897-ebb900e00595'

In [59]:
GP.ggr_get("/api/image/annot/uuid/json", GP.ggr_image_form_arg("c6d2bfe6-629b-dcf3-4329-987968a833ee"))

{'results': [[]], 'success': True}

In [60]:
with open("../data/uuid_gid_map.json", "r") as fl:
    lst = list(json.load(fl).keys())
    
    
with open("../data/ggr_uuid_list.dat", "w") as f2:
    f2.write("\n".join(lst))

In [19]:
uuid_gid_map = {GP.ggr_get("/api/image/uuid/?gid_list=[%i]" %gid)['results'][0]["__UUID__"] : gid for gid in range(1,40811)}

KeyboardInterrupt: 

In [1]:
import requests
import UploadAndDetectIBEIS as UD

In [ ]:
data_dict = {
        'aid_list': [1],
    }

data_dict_2 = {
        'aid_list' : [2,3,4,5,6]
}

annot_uuid_list = get('api/annot/uuid', data_dict)
d_annot_uuid_list = tmux 

url = "http://pachy.cs.uic.edu:5001/api/engine/query/graph/"
data_dict = {
        'query_annot_uuid_list' : json.dumps([annot_uuid_list[0]])
    }
response = requests.request('POST', url, data=data_dict)

In [2]:
gid = UD.upload("/Users/sreejithmenon/Downloads/1.png")

Uploading 1.png


In [8]:
gid = UD.upload("/Users/sreejithmenon/Downloads/grevys.jpg")

Uploading grevys.jpg


In [9]:
UD.run_detection_task(3)

image_uuid 100179933493040723424924051926533554515

Annot aid_list    = [4]
      bboxes      = [[134, 51, 339, 229]]
      thetas      = [0.0]
      species     = ['zebra_grevys']
      confidences = [0.7388371229171753]
      notes       = ['cnnyolodetect']

Deleting aid_list = [4]
      bboxes      = [None]
      thetas      = [None]
      species     = ['____']
      confidences = [None]
      notes       = [None]

Engine Job ID     = 'jobid-0003'
	 Checking status...
	 Checking status...
	 Checking status...
	 Checking status...
	 Checking status...
	 Checking status...
	 Checking status...
	 Checking status...
	 Checking status...
	 Checking status...
	 Checking status...
	 Checking status...
	 Checking status...
	 Checking status...
	 Checking status...
	 Checking status...
	 Checking status...
	 Checking status...
	 Checking status...
	 Checking status...
	 Checking status...
	 Checking status...
	 Checking status...
	 Checking status...
	 Checking status...
	 Checking status..

In [10]:
l = [1,2,3,4,5,6]

In [12]:
l.remove(1)

In [73]:
import json, re
import importlib
importlib.reload(UD)

<module 'UploadAndDetectIBEIS' from '/Users/sreejithmenon/Google Drive/CodeBase/AWESOME/script/UploadAndDetectIBEIS.py'>

In [50]:
data_dict = {
        'aid_list': [i for i in range(1,3306)],
    }
daid_annot_list = UD.get('api/annot/uuid', data_dict)
daid_annot_list

[{'__UUID__': 'c79850f8-f4f9-4557-8091-d742bb2cd501'},
 {'__UUID__': 'ce2802a4-e805-47b6-a4f4-a8ecdf97c3c6'},
 {'__UUID__': '0bfaf2f9-0bed-480d-b7ee-afae456b55cc'},
 {'__UUID__': '9584654d-2079-446b-9543-3e8407f9e8e4'},
 {'__UUID__': '494c9698-ca26-4aa2-8ec6-a5cff35b7210'},
 {'__UUID__': 'bd073849-c5d5-48b2-a448-55553a9d4c3b'},
 {'__UUID__': '31206571-b141-487a-a3f2-5affba577198'},
 {'__UUID__': 'abf32865-48ca-4c67-b87f-f8d86f2036fb'},
 {'__UUID__': '29a75dab-c4ac-4116-b849-e8329518b963'},
 {'__UUID__': '927f6330-4de3-4064-b1c2-3ffe18e417f0'},
 {'__UUID__': '47ee11e6-76ad-4ee9-a468-1cf7ff87fe6a'},
 {'__UUID__': '30efd587-c409-496e-9864-a8b2b0646470'},
 {'__UUID__': '212c066d-9df9-4725-8820-b95b9e1768f8'},
 {'__UUID__': '6dd8e18d-0326-4dec-84aa-33092330984c'},
 {'__UUID__': 'd78c26bb-aff2-483b-ad60-547b6254bb77'},
 {'__UUID__': '0625b251-2dfe-49e0-ae65-6f8464f27694'},
 {'__UUID__': '470a7f7a-30ea-4893-930f-d788c75f65e5'},
 {'__UUID__': 'b9270764-5e65-4d0b-88ab-a323b410db9e'},
 {'__UUID_

In [83]:
 data_dict = {
        'aid_list': [1,2,3,4],
    }
annot_uuid_list = UD.get('api/annot/uuid', data_dict)
annot_uuid_list

[{'__UUID__': '0b69cf48-f7f8-47f8-8d0e-4c66c028ff81'},
 {'__UUID__': '9bb949b5-5802-4d38-98e5-04b9aadc540b'},
 {'__UUID__': 'cb87ae26-d2e3-47fa-83fb-216aebf5b2f2'},
 {'__UUID__': '30865271-2dc1-451c-9964-ef5e87a40530'}]

In [21]:
annot_uuid_list[0]

{'__UUID__': 'd3443911-1700-4c28-bd8d-650fa2df3e74'}

In [42]:
url = "http://pachy.cs.uic.edu:5001/api/engine/query/graph/"
data_dict = {
        'query_annot_uuid_list' : json.dumps([annot_uuid_list[0]]),
        'database_annot_uuid_list' : json.dumps([daid_annot_list[0], daid_annot_list[1], daid_annot_list[3]])
    }
response = requests.request('POST', url, data=data_dict)

In [47]:
while not UD.check_job_status('jobid-0005'):
    print(blah)

In [48]:
data_dict = {
        'jobid' : 'jobid-0005'
    }

result = UD.get("api/engine/job/result", data_dict)['json_result']['inference_dict']['cluster_dict']

In [33]:
name = None
for i in range(len(result['orig_name_list'])):
    if 'NEWNAME' not in result['orig_name_list'][i]:
        # this i has been previously assigned a name
        name = result['orig_name_list'][i]

In [41]:
uuid_aid_map = {'d3443911-1700-4c28-bd8d-650fa2df3e74' : 1,
                'e47f0b75-5803-496d-8175-58feddf1abbb': 2,
                'fd0ec1f5-e9bf-43ed-99b7-822582f28fb9' : 3,
               'b6446a8a-769a-481a-b471-6bf7fe65bd3b' : 4}

In [38]:

    
name

'1'

In [39]:
for annot_uuid_dict in result['annot_uuid_list']:
        aid = uuid_aid_map[annot_uuid_dict["__UUID__"]]

        data_dict = {
            "aid_list" : [aid],
            "name_list" : [name]
        }

        UD.put("api/annot/name", data_dict)

In [49]:
result

{'annot_uuid_list': [{'__UUID__': 'd3443911-1700-4c28-bd8d-650fa2df3e74'},
  {'__UUID__': 'b6446a8a-769a-481a-b471-6bf7fe65bd3b'}],
 'error_flag_list': [['merge'], ['merge']],
 'exemplar_flag_list': [True, False],
 'new_name_list': ['NEWNAME_2', 'NEWNAME_2'],
 'orig_name_list': ['1', 'NEWNAME_-4']}

In [51]:
len(daid_annot_list)

3305

In [84]:
uuid_aid_map = {annot_uuid_list[i-1]['__UUID__'] : i for i in range(1,5)}

In [55]:
daid_annot_list[1]['__UUID__']

'ce2802a4-e805-47b6-a4f4-a8ecdf97c3c6'

In [85]:
with open("/tmp/test.json", "w") as fl:
    json.dump(uuid_aid_map, fl, indent = 4)

In [60]:
import GetPropertiesAPI as GP

In [61]:
GP.getImageFeature([1,2,3,4], "species/text")

['zebra_grevys', 'zebra_grevys', 'giraffe_reticulated', 'zebra_grevys']

In [80]:
data_dict = {
        'nid_list': [1,2,3],
    }
UD.delete('api/name', data_dict)

!!! FAILED REQUEST !!!
	 URL      = 'http://pachy.cs.uic.edu:5001/api/name/'
	 METHOD   = <function delete at 0x10493a9d8>
	 RESPONSE = {"status": {"cache": -1, "message": "API error, Python Exception thrown: <!!! EXCEPTION !!!>\nTraceback (most recent call last):\n  File \u001b\"/opt/ibeis/ibeis/ibeis/control/controller_inject.py\"\u001b, line \u001b317\u001b, in translate_ibeis_webcall\n    output = func(ibs=ibs, **kwargs)\n\u001bTypeError\u001b: delete_names() got an unexpected keyword argument 'nid_list'\n\n[!translate_ibeis_webcall] Error in translate_ibeis_webcall\nExpected Function Definition: delete_names(ibs, name_rowid_list, safe=True, strict=False, verbose=False)\nReceived Function Definition: delete_names(<IBEISController(TEST_DB_SREEJITH) at 0x763dd90>, nid_list=[1, 2, 3])\nkwargs = {'nid_list': [1, 2, 3]}\nargs = ()\nflask.request.args = ImmutableMultiDict([])\nflask.request.form = ImmutableMultiDict([('nid_list', u'[1, 2, 3]')])\n<type 'exceptions.TypeError'>: delete_nam

AssertionError: 

In [81]:
for i in range(1,4):
    UD.run_detection_task(i)

image_uuid 340274514441647462903972532459083121864

Annot aid_list    = [1, 2]
      bboxes      = [[11, 116, 418, 341], [469, 193, 441, 448]]
      thetas      = [0.0, 0.0]
      species     = ['zebra_grevys', 'zebra_grevys']
      confidences = [0.44952255487442017, 0.5022123456001282]
      notes       = ['cnnyolodetect', 'cnnyolodetect']

Deleting aid_list = [1, 2]
      bboxes      = [None, None]
      thetas      = [None, None]
      species     = ['____', '____']
      confidences = [None, None]
      notes       = [None, None]

Engine Job ID     = 'jobid-0007'
	 Checking status...
	 Checking status...
Engine Detections = [{'height': 341, 'xtl': 11, 'ytl': 116, 'confidence': 0.4495, 'theta': 0.0, 'class': 'zebra_grevys', 'width': 418}, {'height': 448, 'xtl': 469, 'ytl': 193, 'confidence': 0.5022, 'theta': 0.0, 'class': 'zebra_grevys', 'width': 441}]

Annot aid_list    = [1, 2]
      bboxes      = [[11, 116, 418, 341], [469, 193, 441, 448]]
      thetas      = [0.0, 0.0]
      sp

In [86]:
UD.run_id_pipeline(range(1,4), 'zebra_grevys')

Extracting uuid for all annotations specified by GID range
Running ID detection for aid 1
Query submitted..!
Job ID: jobid-0011
Waiting for job completion..!
Query complete..!
IDing complete for AID 4 

Running ID detection for aid 2
Query submitted..!
Job ID: jobid-0012
Query complete..!
IDing complete for AID 2 

Running ID detection for aid 4
Query submitted..!
Job ID: jobid-0013
Waiting for job completion..!
Waiting for job completion..!
Query complete..!
IDing complete for AID 4 

